Perform blocks in parallel:

    for site in sites:
        Download 5-minute data    
        Append data to regional dataframe
    With regional dataframe:
        Calculate sea-level-normalized temperature for each site
        Write sea-level-normalized temperature to dataframe

    Download elevation data
    With elevation data:
        With rasterio, load into raster
        Downscale as necessary
    
    Download RAP data (optional)


With elevation numpy array:  
    Create new np array with the same dimensions as the elevation data  
    Use a linear (or other) interpolation method on sea-level-normalized temperature at each pixel  
    Optionally correct/weight points far away from sensors with sea-level-normalized RAP data  
    Print numpy array to raster output (optional for debugging)  
    numpy array calculation; use sea-level temperature and altitude arrays to determine surface temperature at each pixel
    Print temperature array to raster output (final output)  
    Print an uncertainty array  
    Upload outputs to S3  
  
    
    
  

In [1]:
import json
import urllib

import metpy.calc
import numpy as np
import pandas as pd
import pint
import scipy.interpolate

from metpy.units import units
import rasterio

def make_item(json):
    '''
    Takes a json object and creates an items' dataframe
    '''
    df = pd.DataFrame(columns=["siteName", "siteLat", "siteLon", "siteAltitudeM", "temperature", "potential_temperature"])
    for site in json:
        if "temperature" in json[site]:
            df = df.append({"siteName": site, 
                            "siteLat": 44.2706,
                            "siteLon": -71.3033,
                            "siteAltitudeM": 999 * units.meters, 
                            "temperature": float(json[site]["temperature"]) * units.degF,
                            "potential_temperature": units.degF
                           }, 
                           ignore_index=True)
    return df

def get_potential_temperature(df):
    for i, row in df.iterrows():
        if "pressure" not in row:
            pressure = metpy.calc.height_to_pressure_std(row["siteAltitudeM"])
        else:
            pressure = row["pressure"]
            
        if "dewpoint" in row:
            # calculate equivalent potential temperature
            potential_temperature = metpy.calc.equivalent_potential_temperature(pressure, row["temperature"], row["dewpoint"])
        else:
            # calculate potential temperature

            potential_temperature = metpy.calc.potential_temperature(pressure, row["temperature"])
        df.at[i,'potential_temperature'] = potential_temperature.to(units.degF)
        
            
# Should be as simple as pd.read_json once we figure out the callback issue
#pd.read_json('http://xmountwashington.appspot.com/mmNew.php')

In [2]:
import os
import urllib.request
import zipfile

def download_file(url, target, clobber=False):
    # TODO; move this to an S3 call
    make_dirs_if_no_exists(target)
    print("Downloading file %s" % url)
    if os.path.exists(target) and not clobber:
        print("Path %s already exists; using previoiusly downloaded file." % target)
        return False
    urllib.request.urlretrieve(url, target)  
    return True

        
def extract_file(source, target, delete_source=False):
    make_dirs_if_no_exists(target)
    print("Unzipping file %s to %s." % (source, target))
    with zipfile.ZipFile(source,"r") as zip_ref:
        zip_ref.extractall(target)
    

def make_dirs_if_no_exists(dirname):
    if not os.path.exists(os.path.dirname(dirname)):
        print("Path %s does not exist.  Attempting to make it now." % os.path.dirname(dirname))
        os.makedirs(os.path.dirname(dirname))

In [3]:
# Configuration
import os
cwd = os.getcwd()

MESONET_DATA_URL = "http://xmountwashington.appspot.com/mmNew.php?callback=null"

TARGET_DEM_URL = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/13/ArcGrid/USGS_NED_13_n45w072_ArcGrid.zip"
TARGET_DEM_FILE_ZIP = os.path.join(cwd, "tmp/dem.zip")
TARGET_DEM_FILE_UNZIP = os.path.join(cwd, "tmp/data/dem")
TARGET_DEM_DATA_FILE = os.path.join(TARGET_DEM_FILE_UNZIP, "grdn45w072_13", "w001001.adf")



In [4]:
# Driver for downloading mesonet data and performing point calculations

with urllib.request.urlopen(MESONET_DATA_URL) as response:
    callback = response.read()
json_string = callback.decode("utf8").rstrip(')').lstrip('null(')
mesonet_data = json.loads(json_string)
df = make_item(mesonet_data['currentMesonetConditions'])
get_potential_temperature(df)


In [5]:
# Driver for download and extraction of DEM data

if download_file(TARGET_DEM_URL, TARGET_DEM_FILE_ZIP):
    extract_file(TARGET_DEM_FILE_ZIP, TARGET_DEM_FILE_UNZIP)

Path /external/mesonet-interpolation/temperature/tmp/dem.zip already exists; using previoiusly downloaded file.


In [ ]:
# Driver that combines mesonet and DEM data

with rasterio.open(TARGET_DEM_DATA_FILE) as dem:
    #dem_arr = dem.read()
    x = np.arange(0,dem.width)
    y = np.arange(0,dem.height)
    X, Y = np.meshgrid(x,y)
    # Make an array the same size as this array
    #potential_temperature_arr = np.squeeze(np.empty_like(dem_arr))
    
    
    x_pixels = []
    y_pixels = []
    values = []
    
    for i, row in df.iterrows():
        site_y_pixel, site_x_pixel = dem.index(row["siteLon"], row["siteLat"])    
        x_pixels.append(site_x_pixel)
        y_pixels.append(site_y_pixel)
        values.append(row["potential_temperature"].magnitude)
    print(np.asarray(x_pixels).shape)
    print(np.asarray(y_pixels).shape)
    print(x_pixels)
    print(y_pixels)
    print(X)
    print(Y)
    print(np.asarray(values).shape)
    

scipy.interpolate.griddata((np.asarray(x_pixels), np.asarray(y_pixels)), np.asarray(values), (X, Y), method="nearest")



(17,)
(17,)
[7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530, 7530]
[7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883, 7883]
[[    0     1     2 ... 10809 10810 10811]
 [    0     1     2 ... 10809 10810 10811]
 [    0     1     2 ... 10809 10810 10811]
 ...
 [    0     1     2 ... 10809 10810 10811]
 [    0     1     2 ... 10809 10810 10811]
 [    0     1     2 ... 10809 10810 10811]]
[[    0     0     0 ...     0     0     0]
 [    1     1     1 ...     1     1     1]
 [    2     2     2 ...     2     2     2]
 ...
 [10809 10809 10809 ... 10809 10809 10809]
 [10810 10810 10810 ... 10810 10810 10810]
 [10811 10811 10811 ... 10811 10811 10811]]
(17,)


In [ ]:
df['potential_temperature']